In [1]:
import json
import os
import sys
import warnings

import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

In [2]:
sys.path.append(os.path.join('..', 'src'))

In [3]:
import importlib
import model
importlib.reload(model)

from model import get_model_params, timer, measure_prediction_time, apply_ml_model, save_model_parameters, save_model_metrics

# set model parameters and capture data

In [4]:
# scoring = 'neg_mean_squared_error'
scoring = 'f1'

inputs = os.path.join('..', 'data', '03_processed')
models_reports = os.path.join('..', 'data', '04_models')
model_outputs = os.path.join('..', 'data', '05_model_output')
reports = os.path.join('..', 'data', '06_reporting')

X_train            = pd.read_csv(os.path.join(inputs, 'X_train.csv'), index_col='id')
X_train_onehot         = pd.read_csv(os.path.join(inputs, 'X_train_onehot.csv'), index_col='id')
y_train            = pd.read_csv(os.path.join(inputs, 'y_train.csv'), index_col='id')

data_list = [X_train, X_train_onehot, y_train]

for df in data_list:
    print(df.shape)

(7000, 46)
(7000, 158)
(7000, 1)


In [5]:
X_train_onehot.head()

,addr1,addr2,c1,c10,c11,c12,c13,c14,c2,c3,...,m1,m2,m3,m5,m6,m7,m8,m9,transactionamt,transactiondt
id,,,,,,,,,,,,,,,,,,,,,
3486774,270.239972,86.995213,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.999962,0.900851,0.765759,0.520700,0.497537,0.124312,0.507226,0.965394,38.056,13107389.0
3062695,194.000000,87.000000,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.999808,0.918390,0.835247,0.491038,0.463338,0.110642,0.328999,0.869981,150.000,1650884.0
3273443,502.000000,87.000000,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.000000,1.000000,1.000000,0.000000,0.342737,0.000000,0.000000,1.000000,56.500,7048761.0
3384445,300.433901,86.977691,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.999945,0.833067,0.773382,0.533395,0.454835,0.116979,0.444016,0.857350,8.459,10011292.0
3489059,315.000000,87.000000,4.0,0.0,3.0,0.0,8.0,4.0,4.0,0.0,...,1.000000,0.000000,0.000000,0.345176,1.000000,0.000000,0.000000,0.000000,77.950,13159069.0


# Machine Learning

convergence warning: https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge

In [6]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

In [7]:
ml_dict = {}

# Specify the hyperparameter space
# if target_type == 'regression':
#     parameters = {
#     'model__alpha': np.linspace(0.2, 1, 5), 
#     'model__l1_ratio': np.linspace(0, 1, 5),
#     'model__random_state':[42]
#     }
#     ml_model = ElasticNet()
#     # set tol, default is 1e-4
#     do_transform_label = 'log'
# elif target_type == 'binary':
c_space = np.logspace(-5, 1, 5)
parameters = {
'model__C': c_space, 
'model__penalty': ['l2'],
'model__random_state':[42]
}
ml_model = LogisticRegression()
do_transform_label = None

# key = 'standard'

## test with different preprocessing steps
There are 2 different X_sets: On X_train_onehot, I applied one-hot encoding, while on X_train I applied Ordinal Encoding. The former is aimed at linear regression models, and the later is generally used for tree models.

On 'column' parameter, I am able to choose column groups. For instance, I might exclude collinear variables obtained from the VIF function applied on notebook 5. That is useful for linear regression models.

```python
treat_collinearity = False, do_build_polynomals=False, do_treat_skewness=False
```

In [8]:
model_type = 'reg'
ml_dict[model_type] = {}
columns = X_train_onehot.columns

clf, ml_dict[model_type]['train_time'], ml_dict[model_type]['prediction_time'] = apply_ml_model(
    X_train_onehot, y_train, columns, ml_model, parameters, scoring,
    do_build_polynomals=False, 
    do_treat_skewness=False,
    imputation=Imputer(strategy='median'), scaler=StandardScaler(),
    )
ml_dict[model_type]['best_params'], ml_dict[model_type]['best_score']  = get_model_params(clf, scoring)
pprint(ml_dict)

save_model_parameters(models_reports, model_type, clf)
save_model_metrics(model_outputs, model_type, ml_dict)

test type: False
{'reg': {'best_params': {'model__C': 10.0,
                         'model__penalty': 'l2',
                         'model__random_state': 42},
         'best_score': 0.12536338274396502,
         'prediction_time': 0.0003,
         'train_time': 6.464915}}


```python
treat_collinearity = True, do_build_polynomals=False, do_treat_skewness=False,
```

In [9]:
model_type = 'reg_nocol'
ml_dict[model_type] = {}

# columns_nocol = dfs_dict['X_train_oh_nocol'].columns.to_list()

clf, ml_dict[model_type]['train_time'], ml_dict[model_type]['prediction_time'] = apply_ml_model(
    X_train_onehot, y_train, columns, ml_model, parameters, scoring,
    do_build_polynomals=False, 
    do_treat_skewness=False,
    imputation=Imputer(strategy='median'), scaler=StandardScaler(),
    )
ml_dict[model_type]['best_params'], ml_dict[model_type]['best_score']  = get_model_params(clf, scoring)
pprint(ml_dict)

save_model_parameters(models_reports, model_type, clf)
save_model_metrics(model_outputs, model_type, ml_dict)

test type: False
{'reg': {'best_params': {'model__C': 10.0,
                         'model__penalty': 'l2',
                         'model__random_state': 42},
         'best_score': 0.12536338274396502,
         'prediction_time': 0.0003,
         'train_time': 6.464915},
 'reg_nocol': {'best_params': {'model__C': 10.0,
                               'model__penalty': 'l2',
                               'model__random_state': 42},
               'best_score': 0.12536338274396502,
               'prediction_time': 0.0003999,
               'train_time': 5.743999}}


I might use the alternative encoding just to demonstrate the impact on the score.